# Building a Lead Scoring Model with EvalML

In this demo, we will build an optimized lead scoring model using EvalML. To optimize the pipeline, we will set up an objective function to maximize the revenue generated with true positives while taking into account the cost of false positives. At the end of this demo, we also show you how introducing the right objective during the training is over 6x better than using a generic machine learning metric like AUC.

In [1]:
import evalml
from evalml import AutoClassificationSearch
from evalml.objectives import LeadScoring

## Configure LeadScoring 

To optimize the pipelines toward the specific business needs of this model, you can set your own assumptions for how much value is gained through true positives and the cost associated with false positives. These parameters are

* `true_positive` - dollar amount to be gained with a successful lead
* `false_positive` - dollar amount to be lost with an unsuccessful lead

Using these parameters, EvalML builds a pileline that will maximize the amount of revenue per lead generated.

In [2]:
lead_scoring_objective = LeadScoring(
    true_positives=1000,
    false_positives=-10
)

## Dataset

We will be utilizing a dataset detailing a customer's job, country, state, zip, online action, the dollar amount of that action and whether they were a successful lead.

In [3]:
from urllib.request import urlopen
import pandas as pd

customers_data = urlopen('https://featurelabs-static.s3.amazonaws.com/lead_scoring_ml_apps/customers.csv')
interactions_data = urlopen('https://featurelabs-static.s3.amazonaws.com/lead_scoring_ml_apps/interactions.csv')
leads_data = urlopen('https://featurelabs-static.s3.amazonaws.com/lead_scoring_ml_apps/previous_leads.csv')
customers = pd.read_csv(customers_data)
interactions = pd.read_csv(interactions_data)
leads = pd.read_csv(leads_data)

X = customers.merge(interactions, on='customer_id').merge(leads, on='customer_id')
y = X['label']

X = X.drop(['customer_id', 'date_registered', 'birthday','phone', 'email',
        'owner', 'company', 'id', 'time_x',
        'session', 'referrer', 'time_y', 'label'], axis=1)

display(X.head())

,job,country,state,zip,action,amount
0,"Engineer, mining",NaN,NY,60091.0,page_view,NaN
1,"Psychologist, forensic",US,CA,NaN,purchase,135.23
2,"Psychologist, forensic",US,CA,NaN,page_view,NaN
3,Air cabin crew,US,NaN,60091.0,download,NaN
4,Air cabin crew,US,NaN,60091.0,page_view,NaN


## Search for best pipeline

In order to validate the results of the pipeline creation and optimization process, we will save some of our data as a holdout set

EvalML natively supports one-hot encoding and imputation so the above `NaN` and categorical values will be taken care of.

In [4]:
X_train, X_holdout, y_train, y_holdout = evalml.preprocessing.split_data(X, y, test_size=0.2, random_state=0)

print(X.dtypes)

job         object
country     object
state       object
zip        float64
action      object
amount     float64
dtype: object


Because the lead scoring labels are binary, we will use `AutoClassificationSearch`. When we call `.search()`, the search for the best pipeline will begin. 

In [5]:
automl = AutoClassificationSearch(objective=lead_scoring_objective,
                                  additional_objectives=['auc'],
                                  max_pipelines=5,
                                  optimize_thresholds=True)

automl.search(X_train, y_train)

*****************************
* Beginning pipeline search *
*****************************

Optimizing for Lead Scoring. Greater score is better.

Searching up to 5 pipelines. 


FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

✔ XGBoost Binary Classification Pipel...    20%|██        | Elapsed:00:15
✔ Random Forest Binary Classification...    40%|████      | Elapsed:00:33
✔ Logistic Regression Binary Pipeline:      60%|██████    | Elapsed:00:40
✔ XGBoost Binary Classification Pipel...    80%|████████  | Elapsed:00:53
✔ XGBoost Binary Classification Pipel...   100%|██████████| Elapsed:01:07
✔ Optimization finished                    100%|██████████| Elapsed:01:07


### View rankings and select pipeline

Once the fitting process is done, we can see all of the pipelines that were searched, ranked by their score on the lead scoring objective we defined

In [6]:
automl.rankings

,id,pipeline_name,score,high_variance_cv,parameters
0,0,XGBoost Binary Classification Pipeline,14.028986,False,"{'impute_strategy': 'most_frequent', 'percent_..."
1,2,Logistic Regression Binary Pipeline,13.160174,True,"{'impute_strategy': 'mean', 'penalty': 'l2', '..."
2,4,XGBoost Binary Classification Pipeline,12.519549,True,"{'impute_strategy': 'mean', 'percent_features'..."
3,3,XGBoost Binary Classification Pipeline,12.282739,False,"{'impute_strategy': 'most_frequent', 'percent_..."
4,1,Random Forest Binary Classification Pipeline,9.761290,True,"{'impute_strategy': 'median', 'percent_feature..."


to select the best pipeline we can run

In [7]:
best_pipeline = automl.best_pipeline

### Describe pipeline

You can get more details about any pipeline. Including how it performed on other objective functions.

In [8]:
automl.describe_pipeline(automl.rankings.iloc[0]["id"])

******************************************
* XGBoost Binary Classification Pipeline *
******************************************

Supported Problem Types: Binary Classification
Model Family: XGBoost Classifier
Number of features: 5

Pipeline Steps
1. One Hot Encoder
	 * top_n : 10
2. Simple Imputer
	 * impute_strategy : most_frequent
	 * fill_value : None
3. RF Classifier Select From Model
	 * percent_features : 0.8487792213962843
	 * threshold : -inf
4. XGBoost Classifier
	 * eta : 0.38438170729269994
	 * max_depth : 7
	 * min_child_weight : 1.5104167958569887
	 * n_estimators : 397

Training
Training for Binary Classification problems.
Total training time (including CV): 15.5 seconds

Cross Validation
----------------
             Lead Scoring   AUC # Training # Testing
0                  13.813 0.554   2479.000  1550.000
1                  13.490 0.632   2479.000  1550.000
2                  14.784 0.565   2480.000  1549.000
mean               14.029 0.583          -         -
std  

## Evaluate on hold out

Finally, we retrain the best pipeline on all of the training data and evaluate on the holdout

In [9]:
best_pipeline.fit(X_train, y_train)

Now, we can score the pipeline on the hold out data using both the lead scoring score and the AUC.

In [10]:
best_pipeline.score(X_holdout, y_holdout, objectives=["auc", lead_scoring_objective])

OrderedDict([('AUC', 0.534035962526443), ('Lead Scoring', 10.5932932072227)])

## Why optimize for a problem-specific objective?

To demonstrate the importance of optimizing for the right objective, let's search for another pipeline using AUC, a common machine learning metric. After that, we will score the holdout data using the lead scoring objective to see how the best pipelines compare.

In [ ]:
automl_auc = evalml.AutoClassificationSearch(objective='auc',
                                additional_objectives=[],
                                max_pipelines=5,
                                optimize_thresholds=True)

automl_auc.search(X_train, y_train)

*****************************
* Beginning pipeline search *
*****************************

Optimizing for AUC. Greater score is better.

Searching up to 5 pipelines. 


FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

✔ XGBoost Binary Classification Pipel...    20%|██        | Elapsed:00:09
▹ Random Forest Binary Classification...    40%|████      | Elapsed:00:09

like before, we can look at the rankings and pick the best pipeline

In [ ]:
automl_auc.rankings

In [ ]:
best_pipeline_auc = automl_auc.best_pipeline

# train on the full training data
best_pipeline_auc.fit(X_train, y_train)

In [ ]:
# get the auc and lead scoring score on holdout data
best_pipeline_auc.score(X_holdout, y_holdout,  objectives=["auc", lead_scoring_objective])

When we optimize for AUC, we can see that the AUC score from this pipeline is better than the AUC score from the pipeline optimized for lead scoring. However, the revenue per lead gained was only `$7` per lead when optimized for AUC and was `$45` when optimized for lead scoring. As a result, we would gain up to 6x the amount of revenue if we optimized for lead scoring.

This happens because optimizing for AUC does not take into account the user-specified true_positive (dollar amount to be gained with a successful lead) and false_positive (dollar amount to be lost with an unsuccessful lead) values. Thus, the best pipelines may produce the highest AUC but may not actually generate the most revenue through lead scoring.

This example highlights how performance in the real world can diverge greatly from machine learning metrics.